### Import Libraries and Data

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# Import Data
data = pd.read_csv("https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/house_pricing_data/house_pricing_train.csv")
data.head()  # Print first few rows to verify

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,6840701095,20150403T000000,3,1.00,1740,4400,1.5,0,0,3,...,1740,0,1924,0,98122,47.6059,-122.300,1720,4400,548500.0
1,1025049114,20140717T000000,3,2.25,1270,1566,2.0,0,0,3,...,1060,210,2014,0,98105,47.6647,-122.284,1160,1327,625504.0
2,4025300360,20150326T000000,3,2.00,1130,16875,1.0,0,0,4,...,1130,0,1947,0,98155,47.7489,-122.300,1600,14300,349500.0
3,5536500200,20140918T000000,5,3.50,3760,4857,2.0,0,3,3,...,2820,940,2004,0,98072,47.7398,-122.167,3000,5693,730000.0
4,1245003660,20150321T000000,3,2.00,1470,6000,1.0,0,0,3,...,1090,380,1950,1996,98033,47.6829,-122.202,1880,7799,630000.0


### Data Preparation

In [5]:
# Define categorical features
categorical_features = ['bathrooms', 'condition']

# Inspect data types and unique values for categorical columns
print(data[categorical_features].dtypes)
print("Unique Values:\n",data[categorical_features].apply(lambda x: x.unique()))

# Ensure categorical columns are treated as categories
for col in categorical_features:
    data[col] = data[col].astype('category')

# Encode categorical variables using pd.get_dummies
features = pd.get_dummies(data[categorical_features], drop_first=True, dtype=int)

# Include any numeric columns that are not categorical
features['sqft_living15'] = data['sqft_living15']

# Construct the prepared data set including the dependent variable ('label')
prepared_data = pd.concat([data[['price']], features], axis=1)

# Handle missing values by removing rows with any missing values
prepared_data = prepared_data.dropna()

# Display the shape of the prepared data set
print(prepared_data.shape)
# Display the first few rows of the prepared data set
prepared_data.head()


bathrooms    category
condition    category
dtype: object
Unique Values:
 bathrooms    [1.00, 2.25, 2.00, 3.50, 2.50, ..., 0.50, 0.00...
condition    [3, 4, 5, 1, 2]
Categories (5, int64): [1, 2, ...
dtype: object
(17290, 35)


,price,bathrooms_0.5,bathrooms_0.75,bathrooms_1.0,bathrooms_1.25,bathrooms_1.5,bathrooms_1.75,bathrooms_2.0,bathrooms_2.25,bathrooms_2.5,...,bathrooms_6.5,bathrooms_6.75,bathrooms_7.5,bathrooms_7.75,bathrooms_8.0,condition_2,condition_3,condition_4,condition_5,sqft_living15
0,548500.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1720
1,625504.0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1160
2,349500.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1600
3,730000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3000
4,630000.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1880


### Selection of Training, Validation and Test Data

In [6]:
# Set a random seed for reproducibility
np.random.seed(42)

# Shuffle the data
prepared_data = prepared_data.sample(frac=1).reset_index(drop=True)

# Calculate the number of rows for each dataset
n_total = len(prepared_data)
n_training = int(0.7 * n_total)
n_validation = int(0.20 * n_total)

# Split the features and labels for training, validation, and test
training_data = prepared_data.iloc[:n_training]
validation_data = prepared_data.iloc[n_training:n_training+n_validation]
test_data = prepared_data.iloc[n_training+n_validation:]

# Separating features and labels
training_features = training_data.drop('price', axis=1)
validation_features = validation_data.drop('price', axis=1)
test_features = test_data.drop('price', axis=1)

training_labels = training_data[['price']]
validation_labels = validation_data[['price']]
test_labels = test_data[['price']]

# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)


Training features dimensions: (12103, 34)
Validation features dimensions: (3458, 34)
Test features dimensions: (1729, 34)

Training labels dimensions: (12103, 1)
Validation labels dimensions: (3458, 1)
Test labels dimensions: (1729, 1)


#### Data Export

In [7]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")